<a href="https://colab.research.google.com/github/KoMurase/movie_research_2/blob/master/Preprocessing_Title.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#元データ使う場合
#!cp -r '/content/drive/MyDrive/Movies/data/mojo_japan/' '/content/'
#!cp -r '/content/drive/MyDrive/Movies/data/metadata_filmarks/' '/content/'

In [1]:
#前処理済みデータを使う場合
!cp -r '/content/drive/MyDrive/Movies/preprocessed/' '/content/'

In [2]:
!pip install texthero
!pip install textdistance

     |████████████████████████████████| 1.4MB 11.4MB/s 
     |████████████████████████████████| 245kB 47.0MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp36-none-any.whl size=1434676 sha256=b82ab85d3ca9322bc2eebd1ddfa60ee94d71871bb0d3a08db7165685935a3dde
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [3]:
!pip install japanize_matplotlib
import pandas as pd
import codecs 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import japanize_matplotlib

import warnings

import os
from tqdm import tqdm

     |████████████████████████████████| 4.1MB 12.2MB/s 
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-cp36-none-any.whl size=4120276 sha256=d4757c979e5f0a806a2a8b023070554f5433bee7095a728a330eef7be4388d9c
  Stored in directory: /root/.cache/pip/wheels/b7/d9/a2/f907d50b32a2d2008ce5d691d30fb6569c2c93eefcfde55202
Successfully built japanize-matplotlib


In [4]:
def read_data(dir, encode_type):
  output=pd.DataFrame()
  print(dir)
  files=os.listdir(dir)
  print(files)

  for file_ in files:
    if '2017' or '2018' or '2019' in file_: #収集できているファイル


      path = dir+file_
      print(path)
      with codecs.open(path, "r", encode_type, "ignore") as file:
          d = pd.read_csv(file, delimiter=",")
      
      output = pd.concat([output, d], axis=0)

  return output

In [6]:
prepro_dir='/content/preprocessed/'
train = pd.read_csv(prepro_dir+'train_2017-2018.csv')
test = pd.read_csv(prepro_dir+'test_2019.csv')

In [8]:
df = pd.concat([train, test], axis=0).reset_index(drop=True)

In [9]:
"""metadata_dir='/content/metadata_filmarks/'
mojo_dir='/content/mojo_japan/'

data = read_data(metadata_dir, encode_type='utf-8')
mdata = read_data(mojo_dir, encode_type='shift-jis')
mdata = mdata.rename(columns={'Filmarks URL':'URL'})
df = data.merge(mdata, how='left',on='URL').reset_index(drop=True)
drop_cols=[
           'url','Release Date','タイトル(日本名)','タイトル(英名)', 
           'header情報',  'あらすじ'
]
df = df.drop(drop_cols, axis=1) """

"metadata_dir='/content/metadata_filmarks/'\nmojo_dir='/content/mojo_japan/'\n\ndata = read_data(metadata_dir, encode_type='utf-8')\nmdata = read_data(mojo_dir, encode_type='shift-jis')\nmdata = mdata.rename(columns={'Filmarks URL':'URL'})\ndf = data.merge(mdata, how='left',on='URL').reset_index(drop=True)\ndrop_cols=[\n           'url','Release Date','タイトル(日本名)','タイトル(英名)', \n           'header情報',  'あらすじ'\n]\ndf = df.drop(drop_cols, axis=1) "

In [16]:
df=df[df['Title'].duplicated()==False]
df.shape

(517, 2384)

In [17]:
from nltk.util import ngrams
import texthero as hero
from texthero import preprocessing

def line_ngram(line, n=2):
  words = [w for w in line.split(' ') if len(w) != 0]
  return list(ngrams(words, n))

def clearn_by_hero(x):
    custom_pipeline = [preprocessing.fillna
                   , preprocessing.lowercase
                   , preprocessing.remove_digits
                   , preprocessing.remove_punctuation
                   , preprocessing.remove_diacritics
                   , preprocessing.remove_whitespace
                   , preprocessing.remove_stopwords
                  ]
    return hero.clean(x, custom_pipeline)
    
def create_n_gram(x, flg=False, n=3): 
    if flg==True:
      x = clearn_by_hero(x)
    x = pd.Series(x).map(lambda r: line_ngram(r, n=n))
    return x

In [18]:
from textdistance import jaro

In [19]:
custom_pipeline = [preprocessing.fillna
                   , preprocessing.lowercase
                   , preprocessing.remove_digits
                   , preprocessing.remove_punctuation
                   , preprocessing.remove_diacritics
                   , preprocessing.remove_whitespace
                   , preprocessing.remove_stopwords
                  ]

In [20]:
df['Title'] = hero.clean(df['Title'], custom_pipeline)
table = pd.DataFrame(df['Title'])
title = df['Title'].values
for t in tqdm(title):
  table[f'{t}']=table['Title'].map(lambda x: jaro(t, x))

100%|██████████| 517/517 [00:09<00:00, 54.07it/s]


In [21]:
table

,Title,bohemian rhapsody,ralph breaks internet,secret life pets,dragon ball super broly,star born,banana time night,wonderful family wife life,fantastic beasts crimes grindelwald,invisible witness,kamen rider heisei generations forever,grinch,dr seuss grinch,yo kai watch forever friends,nutcracker four realms,pad man,venom,paul apostle christ,searching,christopher robin,beauty beast,despicable,pirates caribbean dead men tell tales,name,moana,sing,star wars episode viii last jedi,detective conan crimson love letter,doraemon great adventure antarctic kachi kochi,la la land,fate furious,gintama live action movie,want eat pancreas,pokteumon movie choose,mary witch flower,spider man homecoming,memoirs murderer,sekigahara,resident evil final chapter,mumon land stealth,...,last christmas,mary queen scots,yo kai watch jam movie yo kai academy cat hero,suspiria,mortal engines,dance,documentary nogizaka,psycho pass sinners system case crime punishment,prince legend,city hunter,promised land,listen universe,taxi,simple favor,black school rules,child play,ninkyo gakuen,train dragon hidden world,boy erased,front runner,angel fallen,brightburn,ai uta yakusoku nakuhito,king prism shiny seven stars,family strangers,chia danshi,leaving scene,kabaneri iron fortress battle unato,happy death day 2u,glass castle,fighting family,blindspotting,wildlife,marrowbone,welcome marwen,beautiful boy,days entebbe,first purge,human flow,dark hall
0,bohemian rhapsody,1.000000,0.572108,0.520425,0.617531,0.506536,0.538126,0.524284,0.502266,0.457516,0.535002,0.447712,0.483287,0.548459,0.444742,0.536134,0.511765,0.519551,0.506536,0.538126,0.617647,0.378431,0.526907,0.578431,0.698039,0.539216,0.500245,0.532569,0.468542,0.531373,0.522176,0.471863,0.522059,0.572093,0.464986,0.541550,0.573529,0.484314,0.533307,0.528322,...,0.514006,0.531863,0.507518,0.406046,0.575941,0.481046,0.562745,0.524667,0.492911,0.521687,0.570324,0.528758,0.372549,0.522176,0.513344,0.595425,0.426244,0.529412,0.673529,0.436928,0.503595,0.464706,0.534967,0.468799,0.531863,0.649287,0.549271,0.502266,0.555011,0.522876,0.483287,0.426244,0.289216,0.590196,0.494398,0.602456,0.356209,0.449554,0.484314,0.549891
2,ralph breaks internet,0.572108,1.000000,0.585588,0.614389,0.595238,0.554809,0.520269,0.593849,0.541550,0.615043,0.476190,0.596429,0.579365,0.553391,0.584127,0.415873,0.623225,0.595238,0.572108,0.515873,0.601984,0.599465,0.531746,0.498413,0.000000,0.558532,0.509646,0.546791,0.573016,0.484127,0.525397,0.572108,0.543579,0.504513,0.561328,0.589616,0.517460,0.601058,0.577249,...,0.542328,0.552579,0.466435,0.487698,0.696825,0.498413,0.492063,0.552579,0.540446,0.443723,0.558812,0.565079,0.432540,0.484127,0.543915,0.446032,0.474237,0.572698,0.582540,0.591270,0.543651,0.512698,0.530952,0.626984,0.567063,0.561328,0.648474,0.642328,0.525132,0.496032,0.390476,0.481074,0.394841,0.461905,0.515873,0.415751,0.615079,0.499278,0.363492,0.560847
4,secret life pets,0.520425,0.585588,1.000000,0.532997,0.481481,0.520425,0.562144,0.574130,0.531863,0.568082,0.451389,0.425000,0.602543,0.583123,0.401786,0.420833,0.583772,0.556019,0.531863,0.513889,0.547222,0.644361,0.437500,0.000000,0.541667,0.555871,0.586508,0.437903,0.384722,0.530754,0.523333,0.573529,0.496212,0.549156,0.447421,0.529762,0.466667,0.654982,0.465939,...,0.586971,0.529762,0.387034,0.409722,0.607143,0.341667,0.447222,0.566239,0.621795,0.548431,0.445513,0.572685,0.541667,0.513889,0.513558,0.537500,0.361645,0.371667,0.537500,0.444444,0.509722,0.384722,0.430556,0.527381,0.541667,0.454545,0.580128,0.576353,0.407407,0.530754,0.415278,0.501068,0.500000,0.441667,0.598214,0.501068,0.569444,0.596050,0.384722,0.506944
5,dragon ball super broly,0.617531,0.614389,0.532997,1.000000,0.551185,0.429180,0.572862,0.538199,0.482381,0.539991,0.432367,0.552657,0.430642,0.564863,0.510559,0.328986,0.594827,0.372786,0.541006,0.486370,0.509179,0.519501,0.362319,0.574638,0.362319,0.503310,0.574327,0.526087,0.590942,0.420290,0.478261,0.566165,0.445487,0.522805,0.569588,0.490942,0.44

In [22]:
for c in table.columns:
  if c!='Title':
    table[c]=table[c].astype(float)

In [23]:
count=0
movie_series=[]
for c in table.columns:
  
  if c!='Title':
    #print(table[table[c]==1]['Title'].values[0])
    result=table[(table[c]>0.77)&(table[c]!=1)]['Title'].values
    if len(result) >0:
      print(f'-{c}-----')
      print(table[(table[c]>0.77)&(table[c]!=1)]['Title'].values)
    
    movie_series.extend(result)

-wonderful family wife life-----
['wonderful family']
-fantastic beasts crimes grindelwald-----
['fantastic beasts find']
-kamen rider heisei generations forever-----
['kamen rider heisei generations final build ex aid legend riders'
 'kamen rider ex aid true ending'
 'kamen rider heisei generations dr pac man vs ex aid ghost legend rider'
 'kamen rider reiwa first generation']
-yo kai watch forever friends-----
['yo kai watch']
-searching-----
['weathering']
-star wars episode viii last jedi-----
['star wars episode ix rise skywalker']
-detective conan crimson love letter-----
['detective conan zero enforcer' 'detective conan darkest nightmare'
 'detective conan fist blue sapphire']
-pokteumon movie choose-----
['pokteumon movie power us']
-spider man homecoming-----
['spider man far home']
-cars-----
['paris']
-fantastic beasts find-----
['fantastic beasts crimes grindelwald']
-fate stay night heaven feel presage flower-----
['fate stay night heaven feel ii lost butterfly']
-corner w

In [24]:
movie_series=sorted(list(set(movie_series)))
movie_series

['american assassin',
 'american made',
 'american renegades',
 'anthem heart',
 'brightburn',
 'cars',
 'city hunter',
 'city hunter shinjuku private eyes',
 'code geass lelouch rebellion ii transgression',
 'code geass lelouch surrection',
 'corner world',
 'crayon shin chan honeymoon hurricane lost hiroshi',
 'crayon shin chan invasion alien shiriri',
 'dark tower',
 'darkest hour',
 'detective conan crimson love letter',
 'detective conan darkest nightmare',
 'detective conan fist blue sapphire',
 'detective conan zero enforcer',
 'diner',
 'dog journey',
 'dog purpose',
 'doraemon movie nobita treasure island',
 'eiga doraemon nobita getsumen tansaki',
 'equalizer',
 'equals',
 'eureka seven hi evolution',
 'eureka seven hi evolution anemone',
 'family strangers',
 'fantastic beasts crimes grindelwald',
 'fantastic beasts find',
 'fate stay night heaven feel ii lost butterfly',
 'fate stay night heaven feel presage flower',
 'founder',
 'free timeless medley bond',
 'free timeless

In [25]:
word_counter_en = {}
for title in movie_series:
  for word in title.split(' '):
    # dict.get(key, 0)はdictにkeyがあればdict[key]を、なければ0を返すメソッドです
    word_counter_en[word] = word_counter_en.get(word, 0) + 1

In [26]:
word_counter_en

{'academia': 2,
 'aid': 3,
 'alien': 1,
 'american': 3,
 'anemone': 1,
 'anthem': 1,
 'assassin': 1,
 'battleship': 3,
 'beasts': 2,
 'blue': 1,
 'bond': 1,
 'brightburn': 1,
 'build': 1,
 'busan': 1,
 'butterfly': 1,
 'cars': 1,
 'castle': 1,
 'ch': 1,
 'chan': 2,
 'chapter': 4,
 'choose': 1,
 'city': 2,
 'clash': 1,
 'code': 2,
 'comet': 1,
 'conan': 4,
 'corner': 1,
 'crayon': 2,
 'crimes': 1,
 'crimson': 1,
 'cure': 3,
 'dark': 1,
 'darkest': 2,
 'das': 2,
 'day': 1,
 'days': 1,
 'departure': 1,
 'detective': 4,
 'detonation': 1,
 'diner': 1,
 'dog': 2,
 'doraemon': 2,
 'dr': 1,
 'eater': 1,
 'eiga': 1,
 'end': 1,
 'ending': 1,
 'enforcer': 1,
 'episode': 2,
 'equalizer': 1,
 'equals': 1,
 'eureka': 2,
 'evolution': 2,
 'ex': 3,
 'eyes': 1,
 'family': 3,
 'fantastic': 2,
 'far': 1,
 'fate': 2,
 'feel': 2,
 'final': 2,
 'finale': 2,
 'find': 1,
 'first': 1,
 'fist': 1,
 'flower': 1,
 'forever': 2,
 'founder': 1,
 'free': 2,
 'friends': 1,
 'geass': 2,
 'generation': 1,
 'generations

In [43]:
def prepro_title(text):
  text = text.replace(':','')\
             .replace('：','')\
             .replace('劇場版','')\
             .replace('映画','')\
             .replace('\u3000',' ')\
             .replace('　',' ')\
             .replace('?',' ')\
             .replace('？',' ')\
             .replace('/','')\
             .replace('〜','')\
             .replace('スパイダーマンスパイダーバース','スパイダーマン スパイダーバース')\
             .replace('スパイダーマンファー・フロム・ホーム','スパイダーマン ファー・フロム・ホーム')
  return text

In [44]:
df['Title(ja)'] = df['Title(ja)'].dropna().apply(prepro_title)
table_ja = pd.DataFrame(df['Title(ja)'].dropna())

title = df['Title(ja)'].dropna().values
for t in tqdm(title):
  table_ja[f'{t}']=table_ja['Title(ja)'].map(lambda x: jaro(t, x))
  table_ja[f'{t}']=table_ja[f'{t}'].astype(float)

100%|██████████| 516/516 [00:07<00:00, 65.55it/s]


In [45]:
movie_series_ja=[]

for c in table_ja.columns:
  
  if c!='Title(ja)':
    #print(table[table[c]==1]['Title'].values[0])
    result=table_ja[(table_ja[c]>0.70)&(table_ja[c]!=1)]['Title(ja)'].values
    if len(result) >0:
      print(f'-{c}-----')
      print(table_ja[(table_ja[c]>0.70)&(table_ja[c]!=1)]['Title(ja)'].values)
    
    movie_series_ja.extend(result)

-アリー スター誕生-----
['アス']
-ファンタスティック・ビーストと黒い魔法使いの誕生-----
['ファンタスティック・ビーストと魔法使いの旅']
-グリンチ-----
['グリーンブック']
-名探偵コナン から紅の恋歌-----
['名探偵コナン ゼロの執行人' '名探偵コナン 純黒の悪夢' '名探偵コナン 紺青の拳']
-ドラえもん のび太の南極カチコチ大冒険-----
['ドラえもん のび太の宝島' 'ドラえもん のび太の月面探査記']
-ポケットモンスター キミにきめた!-----
['ポケットモンスター みんなの物語']
-スパイダーマン ホームカミング-----
['スパイダーマン ファー・フロム・ホーム' 'スパイダーマン スパイダーバース']
-IT／イット “それ”が見えたら、終わり。-----
['IT／イット THE END “それ”が見えたら、終わり。']
-ローグ・ワンスター・ウォーズ・ストーリー-----
['ハン・ソロ／スター・ウォーズ・ストーリー']
-ファンタスティック・ビーストと魔法使いの旅-----
['ファンタスティック・ビーストと黒い魔法使いの誕生']
- Fatestay night Heaven's Feel I. presage flower-----
[" Fatestay night Heaven's Feel II. lost butterfly"]
-HiGH&LOW THE MOVIE2  END OF SKY-----
['HiGH&LOW THE WORST']
-僕のワンダフル・ライフ-----
['僕のワンダフル・ジャーニー']
-仮面ライダー平成ジェネレーションズ FINAL ビルド&エグゼイドwithレジェンドライダー-----
['仮面ライダー平成ジェネレーションズ Dr.パックマン対エグゼイド＆ゴースト with レジェンドライダー']
-パッセンジャー-----
['パワーレンジャー']
-アサシン クリード-----
['アンクル・ドリュー']
-プリキュアドリームスターズ！-----
['プリキュアスーパースターズ！']
-ジョン・ウィックチャプター2-----
['ジョン・ウィックパラベラム']
-キング・アーサー-----
['キングスマンゴールデン・サークル']
-G

In [46]:
movie_series_ja=sorted(list(set(movie_series_ja)))
movie_series_ja

[" Fatestay night Heaven's Feel I. presage flower",
 " Fatestay night Heaven's Feel II. lost butterfly",
 ' Free! Timeless Medley 約束',
 ' Free! Timeless Medley 絆',
 'ANEMONE／交響詩篇エウレカセブン ハイエボリューション',
 'GODZILLA 怪獣惑星',
 'GODZILLA 星を喰う者',
 'HiGH&LOW THE MOVIE2  END OF SKY',
 'HiGH&LOW THE WORST',
 'IT／イット THE END “それ”が見えたら、終わり。',
 'IT／イット “それ”が見えたら、終わり。',
 'Reゼロから始める異世界生活 Memory Snow',
 'Reゼロから始める異世界生活 氷結の絆',
 'T2 トレインスポッティング',
 'TAXi ダイヤモンド・ミッション',
 'それいけ！アンパンマン かがやけ！クルンといのちの星',
 'それいけ！アンパンマン きらめけ！アイスの国のバニラ姫',
 'それいけ！アンパンマン ブルブルの宝探し大冒険',
 'アサシン クリード',
 'アス',
 'アベンジャーズ／インフィニティ・ウォー',
 'アベンジャーズ／エンドゲーム',
 'アリー スター誕生',
 'アンクル・ドリュー',
 'インクレディブル・ファミリー',
 'キングスマンゴールデン・サークル',
 'キング・アーサー',
 'グリンチ',
 'グリーンブック',
 'グレイテスト・ショーマン',
 'コードギアス 反逆のルルーシュII 叛道（はんどう）',
 'コードギアス 復活のルルーシュ',
 'ジョン・ウィックチャプター2',
 'ジョン・ウィックパラベラム',
 'スパイダーマン スパイダーバース',
 'スパイダーマン ファー・フロム・ホーム',
 'スパイダーマン ホームカミング',
 'ダークタワー',
 'ダーク・スクール',
 'トレイン・ミッション',
 'ドラえもん のび太の南極カチコチ大冒険',
 'ドラえもん のび太の宝島',
 'ドラえもん のび太の月面探査記',
 'ハン・ソロ／スター・ウォーズ・ストーリー

In [75]:
word_counter = {}
for title in movie_series_ja:
  for word in title.split(' '):
    if word != 'THE':
      # dict.get(key, 0)はdictにkeyがあればdict[key]を、なければ0を返すメソッドです
      word_counter[word] = word_counter.get(word, 0) + 1

In [76]:
#word_counter = sorted(word_counter.items())
#sortedで返ってくるのはリスト型なため辞書型に直す
word_counter_={}
for k, v in sorted(word_counter.items(), key=lambda x: -x[1]):
  word_counter_.update({k:v})

word_counter_.pop('')
word_counter = pd.Series(word_counter_)

In [77]:
#2回以上カウントされるものを抽出
word_counter = word_counter[word_counter>1]
word_counter

MOVIE                4
名探偵コナン               4
それいけ！アンパンマン          3
スパイダーマン              3
ドラえもん                3
宇宙戦艦ヤマト2202          3
愛の戦士たち               3
Fatestay             2
night                2
Heaven's             2
Feel                 2
Free!                2
Timeless             2
Medley               2
GODZILLA             2
HiGH&LOW             2
END                  2
IT／イット               2
“それ”が見えたら、終わり。       2
Reゼロから始める異世界生活       2
コードギアス               2
ポケットモンスター            2
仮面ライダー平成ジェネレーションズ    2
僕のヒーローアカデミア          2
機動戦士ガンダム             2
ORIGIN               2
魔法少女リリカルなのは          2
dtype: int64

In [86]:
def boolean_contains_keyword(input_df:pd.DataFrame)->pd.DataFrame:
  output_df = input_df.copy()
  cols=[]
  for keyword in word_counter.index:
    output_df[f'Title(ja)_contains_{keyword}']=output_df['Title(ja)'].dropna().apply(lambda x: 1 if keyword in x else 0)
    cols.append(f'Title(ja)_contains_{keyword}')
  return output_df[cols]

In [96]:
df = pd.concat([train,test], axis=0).reset_index(drop=True)
title_df = boolean_contains_keyword(df)
title_df.sum(axis=0)

Title(ja)_contains_MOVIE                 6.0
Title(ja)_contains_名探偵コナン                4.0
Title(ja)_contains_それいけ！アンパンマン           3.0
Title(ja)_contains_スパイダーマン               3.0
Title(ja)_contains_ドラえもん                 3.0
Title(ja)_contains_宇宙戦艦ヤマト2202           3.0
Title(ja)_contains_愛の戦士たち                3.0
Title(ja)_contains_Fatestay              0.0
Title(ja)_contains_night                 2.0
Title(ja)_contains_Heaven's              2.0
Title(ja)_contains_Feel                  2.0
Title(ja)_contains_Free!                 4.0
Title(ja)_contains_Timeless              2.0
Title(ja)_contains_Medley                2.0
Title(ja)_contains_GODZILLA              2.0
Title(ja)_contains_HiGH&LOW              3.0
Title(ja)_contains_END                   7.0
Title(ja)_contains_IT／イット                2.0
Title(ja)_contains_“それ”が見えたら、終わり。        2.0
Title(ja)_contains_Reゼロから始める異世界生活        0.0
Title(ja)_contains_コードギアス               10.0
Title(ja)_contains_ポケットモンスター             2.0
Title(ja)_

#データの結合

In [97]:
train.shape, test.shape, df.shape

((437, 2384), (167, 2384), (604, 2384))

In [98]:
ALL_DATA = pd.concat([df, title_df], axis=1)
train_df = ALL_DATA[0:len(train)].reset_index(drop=True)
test_df = ALL_DATA[len(train):].reset_index(drop=True)

train_df.shape, test_df.shape

((437, 2411), (167, 2411))

In [99]:
train_df.to_csv('/content/drive/MyDrive/Movies/preprocessed/train_2017-2018.csv', index=False)
test_df.to_csv('/content/drive/MyDrive/Movies/preprocessed/test_2019.csv', index=False)